In [1]:
# Bootstrap minimal deps for this notebook
import sys, subprocess, importlib

def ensure(mod_name: str, pip_name: str = None):
    pip_name = pip_name or mod_name
    try:
        importlib.import_module(mod_name)
    except Exception:
        print(f"Installing {pip_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])

# Attempt to ensure core packages
for mod, pipn in [("ipykernel", "ipykernel"), ("ipywidgets", "ipywidgets"), ("PIL", "Pillow"), ("numpy", "numpy")]:
    ensure(mod, pipn)

# Optionally register kernel (best-effort)
try:
    subprocess.check_call([sys.executable, "-m", "ipykernel", "install", "--user"])
except Exception as e:
    print(f"ipykernel install step skipped: {e}")

print("Dependency bootstrap complete.")

Dependency bootstrap complete.


# Geoglyph Viewer with ipywidgets

Interactive widget-based viewer for geoglyph metadata and images.

## 1. Import Required Libraries

Import necessary libraries including ipywidgets, json, PIL, and IPython.display for image rendering.

In [2]:
import io
import json
import os
import sys
from pathlib import Path
from typing import Optional

import numpy as np
import ipywidgets as widgets
from IPython.display import display
from PIL import Image as PILImage

from format import make_random_crops, make_fixed_crops, make_polygon_thresholds_crops, fill_with_noise

# ============================================================================
# DEFAULT CONSTANTS
# ============================================================================
DEFAULT_SEED = 42
DEFAULT_STRIDE = 32
DEFAULT_WINDOW_SIZE = 128
DEFAULT_NOISE_LEVEL = 0.0
DEFAULT_THRESHOLD = 0.1
DEFAULT_IMAGE_TYPE = "overlay"
DEFAULT_MAX_DISPLAY_CROPS = 20
MAX_COMPARISON_CROPS = 40
CROPS_PER_ROW = 5
GRID_BACKGROUND_COLOR = (240, 240, 240)
IMAGE_KEYS = {"overlay": "overlay_jpeg", "ortho": "ortho_jpeg"}
METADATA_MISSING_VALUE = "?"


## 2. Load and Parse JSON Metadata

Create a function to load and parse the JSON metadata file.

In [3]:
def load_metadata(meta_path: str) -> dict:
    """Load metadata JSON file (minimal checks)."""
    with open(meta_path, "r", encoding="utf-8") as f:
        return json.load(f)


def resolve_image_path(meta_path: str, files: dict, image_type: str = DEFAULT_IMAGE_TYPE) -> str:
    """Resolve image path, preferring requested type."""
    base_dir = os.path.dirname(os.path.abspath(meta_path))
    key = IMAGE_KEYS.get(image_type)
    if key and key in files:
        return os.path.join(base_dir, files[key])
    for k in ("overlay_jpeg", "ortho_jpeg"):
        if k in files:
            return os.path.join(base_dir, files[k])
    raise FileNotFoundError("No JPEG image found in metadata 'files'.")


def _extract_polygon_vertices(metadata: dict) -> Optional[list]:
    """Return list of (x, y) pairs from common keys or None."""
    if not metadata:
        return None
    if "polygon_points" in metadata:
        pts = metadata["polygon_points"]
        if isinstance(pts, list) and pts and isinstance(pts[0], dict) and "exterior" in pts[0]:
            coords = pts[0]["exterior"]
        else:
            coords = pts
    elif "polygon" in metadata:
        coords = metadata["polygon"]
    elif "geometry" in metadata and isinstance(metadata["geometry"], dict):
        coords = metadata["geometry"].get("coordinates")
        if coords and isinstance(coords[0], (list, tuple)) and isinstance(coords[0][0], (list, tuple)):
            coords = coords[0]
    elif "coordinates" in metadata:
        coords = metadata["coordinates"]
    else:
        return None
    try:
        return [(float(x), float(y)) for x, y in coords]
    except Exception:
        return None


def _polygon_geo_to_pixel(vertices: list, metadata: dict) -> Optional[list]:
    """Map lon/lat polygon to pixel using bounds + image_shape when present."""
    b = metadata.get("bounds")
    s = metadata.get("image_shape")
    if not b or not s:
        return vertices
    minx, miny, maxx, maxy = b.get("minx"), b.get("miny"), b.get("maxx"), b.get("maxy")
    w, h = s.get("width"), s.get("height")
    if None in (minx, miny, maxx, maxy, w, h) or w == 0 or h == 0:
        return vertices
    dx = (maxx - minx) or 1.0
    dy = (maxy - miny) or 1.0
    return [((lon - minx) / dx * w, (maxy - lat) / dy * h) for lon, lat in vertices]


def load_polygon_vertices_from_metadata(metadata: dict) -> Optional[list]:
    """Extract polygon vertices and convert to pixels if possible."""
    v = _extract_polygon_vertices(metadata)
    return _polygon_geo_to_pixel(v, metadata) if v else None


def get_image_and_polygon(meta_path: str, image_type: str = DEFAULT_IMAGE_TYPE):
    """Convenience: returns (img_array, polygon_vertices, poly_idx, clazz, image_path)."""
    md = load_metadata(meta_path)
    image_path = resolve_image_path(meta_path, md.get("files", {}), image_type)
    img_array = np.array(PILImage.open(image_path))
    polygon = load_polygon_vertices_from_metadata(md)
    return img_array, polygon, md.get("polygon_index", METADATA_MISSING_VALUE), md.get("class", METADATA_MISSING_VALUE), image_path


## 3. Create Image Display Function

Implement a function that displays the geoglyph image and metadata information.

## 3a. Crop Visualization Helper

Create a function to overlay crop regions on the image.

In [4]:
def _convert_to_uint8_image(crop_array: np.ndarray) -> np.ndarray:
    """Convert array to uint8 and drop alpha if present."""
    if not isinstance(crop_array, np.ndarray):
        return crop_array
    arr = crop_array
    if arr.dtype != np.uint8:
        arr = np.clip(arr, 0, 255).astype(np.uint8)
    if arr.ndim == 3 and arr.shape[2] == 4:
        arr = arr[:, :, :3]
    return arr


def _generate_crops(image_array: np.ndarray, method: str, window_size: int, n_crops: int, seed: int,
                     stride: int, threshold: float, polygon_vertices: Optional[list]):
    if method == "random":
        return make_random_crops(image_array, window_size, n_crops, seed=seed)
    if method == "polygon_threshold":
        if polygon_vertices is None:
            return make_fixed_crops(image_array, window_size, n_crops=n_crops, stride=stride)
        return make_polygon_thresholds_crops(image_array, polygon_vertices, window_size, n_crops=n_crops, stride=stride, threshold=threshold)
    return make_fixed_crops(image_array, window_size, n_crops=n_crops, stride=stride)


def _pick_subset(crops_list, count: int, seed: int):
    if len(crops_list) <= count:
        return crops_list
    rng = np.random.default_rng(seed)
    idx = rng.choice(len(crops_list), size=count, replace=False)
    return [crops_list[i] for i in idx]


def _grid_from_crops(crops_list, window_size: int, noise_type: Optional[str] = None, noise_level: float = 0.0, seed: int = DEFAULT_SEED):
    if not crops_list:
        size = (CROPS_PER_ROW * window_size, window_size)
        img = PILImage.new('RGB', size, color=GRID_BACKGROUND_COLOR)
        from PIL import ImageDraw
        d = ImageDraw.Draw(img)
        text = "No crops"
        bbox = d.textbbox((0, 0), text)
        x = (size[0] - (bbox[2] - bbox[0])) // 2
        y = (size[1] - (bbox[3] - bbox[1])) // 2
        d.text((x, y), text, fill=(100, 100, 100))
        return img
    # Optional noise fill
    if noise_type and noise_level > 0:
        crops_list = [ (fill_with_noise(c, m, noise_level, noise_type, seed=seed + i), m) for i, (c, m) in enumerate(crops_list) ]
    rows = (len(crops_list) + CROPS_PER_ROW - 1) // CROPS_PER_ROW
    grid_size = (CROPS_PER_ROW * window_size, rows * window_size)
    grid = PILImage.new('RGB', grid_size, color=GRID_BACKGROUND_COLOR)
    for i, (crop, _mask) in enumerate(crops_list):
        x = (i % CROPS_PER_ROW) * window_size
        y = (i // CROPS_PER_ROW) * window_size
        grid.paste(PILImage.fromarray(_convert_to_uint8_image(crop)), (x, y))
    return grid


def create_crops_grid(image_array: np.ndarray, crop_method: str, window_size: int = DEFAULT_WINDOW_SIZE,
                      n_crops: int = DEFAULT_MAX_DISPLAY_CROPS, seed: int = DEFAULT_SEED,
                      stride: int = DEFAULT_STRIDE, threshold: float = DEFAULT_THRESHOLD,
                      polygon_vertices: Optional[list] = None, noise_type: Optional[str] = None,
                      noise_level: float = DEFAULT_NOISE_LEVEL) -> PILImage.Image:
    """Generate crops then assemble a grid (concise pipeline)."""
    show_n = min(n_crops, DEFAULT_MAX_DISPLAY_CROPS)
    crops = _generate_crops(image_array, crop_method, window_size, n_crops, seed, stride, threshold, polygon_vertices)
    if crop_method in {"fixed", "polygon_threshold"}:
        crops = _pick_subset(crops, show_n, seed)
    else:
        crops = crops[:show_n]
    return _grid_from_crops(crops, window_size, noise_type, noise_level, seed)


In [5]:
def _pil_to_bytes(pil_img: PILImage.Image) -> bytes:
    """Convert PIL Image to bytes."""
    buf = io.BytesIO()
    pil_img.save(buf, format='JPEG')
    buf.seek(0)
    return buf.read()


def _get_crop_label(crop_method: str, n_crops: int, threshold: float = DEFAULT_THRESHOLD) -> str:
    show = min(n_crops, MAX_COMPARISON_CROPS)
    return (f"Polygon Threshold (threshold: {threshold}, showing {show}/{n_crops})"
            if crop_method == "polygon_threshold" else
            f"{crop_method.capitalize()} Crops (showing {show}/{n_crops})")


def _info_html(poly_idx: str, clazz: str, image_type: str, crop_method: Optional[str] = None,
               window_size: Optional[int] = None, n_crops: Optional[int] = None) -> str:
    ws = f" | <b>Window:</b> {window_size}" if window_size else ""
    nc = f" | <b>Crops:</b> {n_crops}" if n_crops else ""
    cm = f"<p><b>Crop Method:</b> {crop_method}</p>" if crop_method else ""
    return f"""
    <div style=\"border:1px solid #ddd;padding:12px;border-radius:6px;background:#f9f9f9;margin-top:10px;\">
        <h3>Geoglyph #{poly_idx}</h3><hr>
        <p><b>Class:</b> {clazz} | <b>Image:</b> {image_type}{ws}{nc}</p>
        {cm}
    </div>"""


def display_geoglyph_with_crops(meta_path: str, image_type: str = DEFAULT_IMAGE_TYPE,
                                crop_method: Optional[str] = None,
                                window_size: int = DEFAULT_WINDOW_SIZE,
                                n_crops: int = DEFAULT_MAX_DISPLAY_CROPS,
                                seed: int = DEFAULT_SEED, stride: int = DEFAULT_STRIDE,
                                threshold: float = DEFAULT_THRESHOLD,
                                polygon_vertices: Optional[list] = None,
                                noise_type: Optional[str] = None,
                                noise_level: float = DEFAULT_NOISE_LEVEL):
    """Display original + optional crops side by side (concise)."""
    img_array, inferred_polygon, poly_idx, clazz, image_path = get_image_and_polygon(meta_path, image_type)
    with open(image_path, 'rb') as f:
        left_img = widgets.Image(value=f.read(), format='jpg')
    if crop_method == "comparison":
        return display_comparison(meta_path, image_type, img_array, window_size, n_crops, seed, threshold,
                                 polygon_vertices or inferred_polygon, noise_type, noise_level)
    if crop_method:
        grid = create_crops_grid(img_array, crop_method, window_size, n_crops, seed, stride, threshold,
                                 polygon_vertices or inferred_polygon, noise_type, noise_level)
        right_img = widgets.Image(value=_pil_to_bytes(grid), format='jpg', width='100%')
        right_label = _get_crop_label(crop_method, n_crops, threshold)
    else:
        right_img = left_img
        right_label = "Original Image"
    display(widgets.HTML(_info_html(poly_idx, clazz, image_type, crop_method, window_size, n_crops)))
    display(widgets.HBox([
        widgets.VBox([widgets.HTML("<b>Original Image</b>"), left_img], layout=widgets.Layout(flex='1')),
        widgets.VBox([widgets.HTML(f"<b>{right_label}</b>"), right_img], layout=widgets.Layout(flex='1'))
    ], layout=widgets.Layout(width='100%')))


def display_comparison(meta_path: str, image_type: str, img_array: np.ndarray,
                      window_size: int, n_crops: int, seed: int, threshold: float,
                      polygon_vertices: Optional[list],
                      noise_type: Optional[str] = None, noise_level: float = DEFAULT_NOISE_LEVEL):
    """Show original + Random/Fixed/Polygon grids."""
    md = load_metadata(meta_path)
    poly_idx = md.get("polygon_index", METADATA_MISSING_VALUE)
    clazz = md.get("class", METADATA_MISSING_VALUE)
    image_path = resolve_image_path(meta_path, md.get("files", {}), image_type)
    with open(image_path, 'rb') as f:
        original_img = widgets.Image(value=f.read(), format='jpg', width='100%')
    methods = [("Random", "random"), ("Fixed", "fixed"), ("Polygon Threshold", "polygon_threshold")]
    grids = {name: create_crops_grid(img_array, method, window_size, n_crops, seed=seed,
                                    threshold=threshold, polygon_vertices=polygon_vertices,
                                    noise_type=noise_type, noise_level=noise_level)
             for name, method in methods}
    items = [widgets.VBox([widgets.HTML("<b>Original Image</b>"), original_img], layout=widgets.Layout(flex='1'))]
    show = min(n_crops, MAX_COMPARISON_CROPS)
    for name, _ in methods:
        items.append(widgets.VBox([
            widgets.HTML(f"<b>{name} Crops<br>(showing {show}/{n_crops})</b>"),
            widgets.Image(value=_pil_to_bytes(grids[name]), format='jpg', width='100%')
        ], layout=widgets.Layout(flex='1')))
    display(widgets.HTML(_info_html(poly_idx, clazz, image_type, "comparison", window_size, n_crops)))
    display(widgets.HBox(items, layout=widgets.Layout(width='100%')))


def display_geoglyph(meta_path: str, image_type: str = DEFAULT_IMAGE_TYPE):
    display_geoglyph_with_crops(meta_path, image_type)


## 4. Build Interactive Widget Interface

Create ipywidgets controls to navigate and view geoglyphs.

In [6]:
# Widget style constant
WIDGET_STYLE = {"description_width": "120px"}

crop_method_dropdown = widgets.Dropdown(
    options=[("None", None), ("Random", "random"), ("Fixed", "fixed"), ("Polygon Threshold", "polygon_threshold"), ("Comparison", "comparison")],
    value=None,
    description="Crop Method:",
    style=WIDGET_STYLE
)

In [7]:
# Create widgets with consistent styling
metadata_path_input = widgets.Text(
    value="test_geos/unita_geoglif_0000_metadata.json",
    placeholder="Enter metadata file path",
    description="Metadata Path:",
    style=WIDGET_STYLE,
    layout=widgets.Layout(width="100%")
)

image_type_dropdown = widgets.Dropdown(
    options=[("Overlay", "overlay"), ("Ortho", "ortho")],
    value="overlay",
    description="Image Type:",
    style=WIDGET_STYLE
)

window_size_input = widgets.IntSlider(
    value=128, min=32, max=512, step=32,
    description="Window Size:",
    style=WIDGET_STYLE
)

n_crops_input = widgets.IntSlider(
    value=16, min=1, max=100, step=1,
    description="Number of Crops:",
    style=WIDGET_STYLE
)

crop_seed_input = widgets.IntText(
    value=42,
    description="Random Seed:",
    style=WIDGET_STYLE
)

stride_input = widgets.IntSlider(
    value=32, min=1, max=256, step=1,
    description="Stride:",
    style=WIDGET_STYLE
)

threshold_input = widgets.FloatSlider(
    value=0.1, min=0.0, max=1.0, step=0.01,
    description="Threshold:",
    style=WIDGET_STYLE
)

noise_type_dropdown = widgets.Dropdown(
    options=[("None", None), ("Gaussian", "gaussian"), ("Uniform", "uniform")],
    value=None,
    description="Noise Type:",
    style=WIDGET_STYLE
)

noise_level_input = widgets.FloatSlider(
    value=0.0, min=0.0, max=1.0, step=0.1,
    description="Noise Level:",
    style=WIDGET_STYLE
)

load_button = widgets.Button(
    description="Load Geoglyph",
    button_style="info",
    tooltip="Click to load and display the geoglyph"
)

output_area = widgets.Output()

In [8]:
# Define button click handler
def on_load_click(b):
    output_area.clear_output(wait=True)
    with output_area:
        if not metadata_path_input.value.strip():
            print("Please enter a metadata file path.")
            return
        
        display_geoglyph_with_crops(
            metadata_path_input.value,
            image_type=image_type_dropdown.value,
            crop_method=crop_method_dropdown.value,
            window_size=window_size_input.value,
            n_crops=n_crops_input.value,
            seed=crop_seed_input.value,
            stride=stride_input.value,
            threshold=threshold_input.value,
            noise_type=noise_type_dropdown.value,
            noise_level=noise_level_input.value
        )

load_button.on_click(on_load_click)

## 5. Display Geoglyph with Metadata Information

Enter the path to a metadata JSON file and click "Load Geoglyph" to view the image and metadata.

In [9]:
# Layout the widgets
control_box = widgets.VBox([
    metadata_path_input,
    widgets.HBox([image_type_dropdown, load_button]),
    widgets.HTML("<hr><b>Crop Visualization Settings:</b>"),
    crop_method_dropdown,
    widgets.HBox([window_size_input, n_crops_input]),
    widgets.HBox([stride_input, threshold_input]),
    crop_seed_input,
    widgets.HTML("<hr><b>Noise Filling Settings:</b>"),
    widgets.HBox([noise_type_dropdown, noise_level_input])
])

# Display the interface
display(control_box)
display(output_area)

Output()